In [80]:
import __future__
from Weight import Weight
from PartitionTree import PartitionTree
from sage.all import *

def Eval(string):
    return eval(compile(str(string), '<string>', 'eval', __future__.division.compiler_flag))

def convertWeightParameters(name, weight, simple):
    weyl_group = WeylGroup(name, prefix = "s")
    standard_to_simple = getBasisChange(name)
    fund_to_simple = getFundamentalToSimple(name)

    while not len(weight) == standard_to_simple.ncols():

        weight.append(0)
    if not simple:
        print(weight,(fund_to_simple * vector(weight), standard_to_simple.inverse() * (fund_to_simple * vector(weight))))
        weight = standard_to_simple.inverse() * (fund_to_simple * vector(weight))
        
    else:
        weight = standard_to_simple.inverse() * vector(weight)

    return weyl_group.domain()(list(eval(str(weight))))

def changeFundToSimple(name, weight):
    basis_change = getFundamentalToSimple(name)
    weight = list(weight)
    while not len(weight) == basis_change.ncols():
            weight.append(0)
    return basis_change * vector(weight)

def getFundamentalToSimple(name):
    simples = RootSystem(name).ambient_space().simple_roots()
    simple_basis = ([Eval((str(x).replace("(", "[").replace(")", "]"))) for x in simples])
    for i in range(len(simple_basis), len(simple_basis[0])):
        simple_basis.append([1 if j==i else 0 for j in range(0, len(simple_basis[0]))])

    fundamentals = RootSystem(name).ambient_space().fundamental_weights()
    fund_basis = ([Eval((str(x).replace("(", "[").replace(")", "]"))) for x in fundamentals])
    for i in range(len(fund_basis), len(fund_basis[0])):
        fund_basis.append([1 if j==i else 0 for j in range(0, len(fund_basis[0]))])

    return matrix(simple_basis).transpose().inverse() * matrix(fund_basis).transpose()

def getBasisChange(name):
    simples = RootSystem(name).ambient_space().simple_roots()
    basis_change = ([Eval((str(x).replace("(", "[").replace(")", "]"))) for x in simples])
    for i in range(len(basis_change), len(basis_change[0])):
        basis_change.append([1 if j==i else 0 for j in range(0, len(basis_change[0]))])
    return matrix(basis_change).transpose().inverse()


def findAltSet(name, lamb = None, mu = None, simple = True):
    # initialize constants and vector space for the lie algebra
    lie_algebra = RootSystem(name).ambient_space()
    weyl_group = WeylGroup(name, prefix = "s")
    simples = weyl_group.gens()

    altset = [weyl_group.one()]

    # used to change the basis from the standard basis of R^n to simple roots
    basis_change = getBasisChange(name)

    # if lambda is not specified, the highest root is used
    if lamb == None:
        lamb = lie_algebra.highest_root()
    elif type(lamb) is list:
        lamb = convertWeightParameters(name, lamb, simple)

    # if mu is not specified, 0 vector is used
    if mu == None:
        mu = weyl_group.domain()([0 for i in range(0, len(lie_algebra.simple_roots()))])
    elif type(mu) is list:
        mu  = convertWeightParameters(name, mu, simple)

    # check to see if the alt set is the empty set
    init = (lamb + mu)
    init = basis_change * vector(list(Eval(init)))
    init = Weight(init)

    if init.isNegative():
    
        return []

    rho = lie_algebra.rho()
    length = len(altset)
    i=0
    while i < length:
        for simple in simples:
            if ((altset[i] == simple)or (altset[i] == altset[i] * simple)):
                continue
            res = (altset[i]*simple).action(lamb + rho) - (rho + mu)
            res = basis_change * vector(list(Eval(res)))
            res = Weight(res)

            if not (res.isNegative() or res.hasFraction()):
                if not (altset[i]*simple in altset):
                    altset.append(altset[i]*simple)
                    length += 1
        i+=1

    return altset

In [81]:
def geometricSumForPartition(positive_root, translations, q_analog):
    x = 1
    for i in range(0, len(positive_root)):
        for j in range(0, positive_root[i]):
            x = x * translations["A" + str(i+1)]
    return 1/(1 - x) if not q_analog else 1/(1 -translations['q']*x)


def calculatePartition(name, weight, positive_roots, translations, q_analog ):
    root_system = RootSystem(name).ambient_space()
#   여기를 바꿔야됨
#   termsForSum을 for문으로 바꾸고 positive_roots 조건에 따라 translations를 바꿔서 넣어주는 방향
    termsForSum = [geometricSumForPartition(list(x), translations, q_analog) for x in positive_roots]
    answer = 1
    for x in termsForSum:
        answer *= x

    for i in range(0, len(weight)):
        answer = answer.series(translations["A" + str(i+1)], weight[i] + 1).truncate()
        answer = answer.coefficient(translations["A" + str(i+1)], weight[i])

    answer = answer.expand()
    return answer

In [82]:
def calculateMultiplicity(name, lamb = None, mu = None):
    mult, q_analog, simple = 0 , True, False
    lie_algebra = RootSystem(name).ambient_space()
    weyl_group = WeylGroup(name, prefix = "s")
    # used to change the basis from the standard basis of R^n to simple roots
    standard_to_simple = getBasisChange(name)
    fund_to_simple = getFundamentalToSimple(name)

    positive_roots = [vector(list(Eval(x))) for x in RootSystem(name).ambient_space().positive_roots()]
    positive_roots = [standard_to_simple * x for x in positive_roots]

    lamb = convertWeightParameters(name, lamb, simple)
    mu = convertWeightParameters(name, mu, simple)
    rho = lie_algebra.rho()
    altset = findAltSet(name, lamb, mu)

    s = 'q, A1'
    for i in range(1, int(name[1:])):
        s += ', A' + str(i + 1)
    variables = var(s)
    translations = { 'q' : q }         
    for i in range(0, int(name[1:])):
        translations["A" + str(i+1)] = eval("A" + str(i+1))
        
    for elm in altset:
        # expression in partition function
        res = elm.action(lamb + rho) - (mu + rho)

        #change basis from standard basis to simple roots
        res = vector(list(Eval(res)))
        res = standard_to_simple * res

        term = calculatePartition(name, list(res), positive_roots, translations = translations, q_analog=q_analog)

        term *= (-1)**elm.length()
        mult += term

    return mult    

In [83]:
def customMultiplicity(name, lamb = None, mu = None):
    mult, q_analog, simple = 0 , True, False
    lie_algebra = RootSystem(name).ambient_space()
    weyl_group = WeylGroup(name, prefix = "s")
    # used to change the basis from the standard basis of R^n to simple roots
    standard_to_simple = getBasisChange(name)
    fund_to_simple = getFundamentalToSimple(name)

    positive_roots = [vector(list(Eval(x))) for x in RootSystem(name).ambient_space().positive_roots()]   
    positive_roots = [standard_to_simple * x for x in positive_roots]

    lamb = convertWeightParameters(name, lamb, simple)
    mu = convertWeightParameters(name, mu, simple)
    rho = lie_algebra.rho()
    altset = findAltSet(name, lamb, mu)

    s = 'q, A1'
    for i in range(1, int(name[1:])):
        s += ', A' + str(i + 1)
    variables = var(s)
    translations = { 'q' : q }         
    for i in range(0, int(name[1:])):
        translations["A" + str(i+1)] = eval("A" + str(i+1))
        
    for elm in altset:
        # expression in partition function
        res = elm.action(lamb + rho) - (mu + rho)

        #change basis from standard basis to simple roots
        res = vector(list(Eval(res)))
        res = standard_to_simple * res

        term = customPartition(name, list(res), positive_roots, translations = translations)

        term *= (-1)**elm.length()
        mult += term

    return mult    

In [84]:
def customPartition(name, weight, positive_roots, translations):
    q_analog = True
    root_system = RootSystem(name).ambient_space()
    n=binomial(int(name[1:]),2)
    termsForSum=[]
    for i in range(len(positive_roots)):
        if i in range(n):
            translations['q']=q
        elif i in range(n,2*n):
            translations['q']=q
        else:
            translations['q']=q
        termsForSum += [geometricSumForPartition(list(positive_roots[i]),translations,q_analog)]
    answer = 1
    for x in termsForSum:
        answer *= x

    for i in range(0, len(weight)):
        answer = answer.series(translations["A" + str(i+1)], weight[i] + 1).truncate()
        answer = answer.coefficient(translations["A" + str(i+1)], weight[i])

    answer = answer.expand()
    return answer

In [87]:
customMultiplicity("C4",[0,0,0,4],[0,0,0,3])

[0, 0, 0, 4] ((4, 8, 12, 8), (4, 4, 4, 4))
[0, 0, 0, 3] ((3, 6, 9, 6), (3, 3, 3, 3))


q^8 + q^6 + q^4

In [148]:
positive_roots = [vector(list(Eval(x))) for x in RootSystem("C2").ambient_space().positive_roots()]
positive_roots

[(1, 1), (1, -1), (2, 0), (0, 2)]

In [67]:
name="C2"

standard_to_simple = getBasisChange(name)
print(standard_to_simple)

fund_to_simple = getFundamentalToSimple(name)
print(fund_to_simple)

positive_roots1 = [vector(list(Eval(x))) for x in RootSystem(name).ambient_space().positive_roots()]
print(positive_roots1)

positive_roots2 = [standard_to_simple * x for x in positive_roots1]
print(positive_roots2)

standard_to_simple * positive_roots1[0]
print(type(RootSystem(name).ambient_space().positive_roots()[1][1]))
RootSystem(name).ambient_space().positive_roots()[-1]

[  1   0]
[1/2 1/2]
[  1   1]
[1/2   1]
[(1, 1), (1, -1), (2, 0), (0, 2)]
[(1, 1), (1, 0), (2, 1), (0, 1)]
<class 'sage.rings.rational.Rational'>


(0, 2)

In [ ]:
123123123